# Prerequisites

## You need to install bash terminal on your jupyter server by using the following command (if you haven't already ).
This is also mentioned in the debug_server.ipynb notebook. Just uncomment the following lines when executing it to install the bash kernel

In [87]:
# !pip install bash_kernel
# !python -m bash_kernel.install
# !source deactivate py3k

## Please run the debug server by going through the instructions and executing accordingly in the debug_server.ipynb notebook

**ptvsd** is a python module used by Microsoft Visual Studio Code (VSCode) to debug python code. We will be reusing that as our python debug server. 

Obviously, that makes this a dependency. The command for installation is in the debug_server.ipynb notebook. Just uncomment that and run it to install ptvsd


## Terminology

The following terminology will be used in the notebook.

- Debugger - Debug Adapter/Debug Server -> **ptvsd** in our case
- Debuggee - The process that the debugger is debugging
- Debug client - this file that's communicating with the debugger, can be any IDE/editor too

## About Debug Adapter Protocol

Details about Debug Adapter Protocol (DAP) can be found at https://microsoft.github.io/debug-adapter-protocol/overview. It is a json based wire-protocol that communicates over sockets.

DAP requests from the specification supported by ***ptvsd*** in alphabetical order (of https://microsoft.github.io/debug-adapter-protocol/specification ) - 

- Attach
- Completions
- ConfigurationDone
- Continue
- Disconnect
- Evaluate
- ExceptionInfo
- Goto
- GotoTargets
- Initialize
- Launch
- Modules
- Next
- Pause
- Scopes
- SetBreakpoints
- SetExceptionBreakpoints
- SetExpression
- SetVariable
- Source
- StackTrace
- StepBack
- StepIn
- StepOut
- Terminate
- Threads
- Variables

# Programming a communicator class that communicates with the Debugger.

In [88]:
import json                                                                     
import socket                                                                   
from pprint import pprint                                       
import time   
import random

## connectionClass

**connectionClass** handles the client side socket and its connection to the Debugger

The following methods of the class do the following:
- **init** - initialises the socket and connects to the debugger
- **connect** - connects to the debugger
- **parse_all_messages** - takes in the complete message received and parses for individual messages sent from the debugger
- **recvMessage** - reads the socket fd, calls parse_all_messages and pretty prints the output on the screen (for now)
- **sendMessage** - takes in a JSON object, serializes it, adds DAP headers and sends it on the socket fd. Sleeps for one second after sending the message to allow for all responded messages to arrive.
- **doRequest** - adds in required parameters - seq (no) & type to the json obect before calling sendMessage.  
- **close_connection** - Does one last read of the socket fd and closes the socket connection
- **reset_connection** - closes the connection and then opens up a new connection to the debugger

In [89]:
class connectionClass:
    
    def __init__(self, hostname='localhost', port=5678, timeout=5, sock=None):
        self.msg_id = random.randint(1, 500) #each request increments the value
        self.connection_no = 1
        self.mysocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.port = port
        self.hostname = hostname
        self.mysocket.settimeout(timeout)
        self.connect()
    
        
    def connect(self):
        self.mysocket.connect((self.hostname, self.port))
        self.connection_no += 1
        time.sleep(1)
        #the debug server responds to successful connection
        self.recvMessage("connection no {} ".format(self.connection_no))
    
    def parse_all_messages(self, msg_bytes):
        msg_str = msg_bytes.decode('utf-8')
        split_delimiter = 'Content-Length: '
        msg_list = msg_str.split(split_delimiter)
        if (msg_list[0] == ''):
            msg_list = msg_list[1:]
        parsed_msg_list = [msg.split('\r\n\r\n') for msg in msg_list]
        for index in range(len(parsed_msg_list)):
            parsed_msg_list[index][0] = split_delimiter + parsed_msg_list[index][0]  
        return parsed_msg_list

    #for now, this function just prints the received msg, 
    #it can be modified to return the data
    def recvMessage(self, func_name):
        sock = self.mysocket
        try: 
            recv_data = sock.recv(1024*1024)
        except socket.timeout:                                                          
            print ("Timeout. Nothing was received at", func_name)
            return
        list_of_msgs = self.parse_all_messages(recv_data)
        for msg in list_of_msgs:
            print ("Received data:")
            print (msg[0])
            try:
                pprint(json.loads(msg[1]))
            except:
                print ("json.loads failed in recvMessage, printing entire message")
                print(msg)
            print()
    
    def sendMessage(self, msg):
        sock = self.mysocket
        s_msg = json.dumps(msg)                                                       
        data = 'Content-Length: {0}\r\n\r\n{1}'.format( len( s_msg ), s_msg )
        print ('Sending data: Content-Length: {0}'.format(len(s_msg)))
        pprint(msg)
        print()
        data = bytes(data, 'utf-8')                                                   
        sock.send(data)                                                                  
        time.sleep(1)
    
    def doRequest(self, msg):                                                                                                                  
        this_id = self.msg_id                                                     
        self.msg_id += 1                                                          
        msg[ 'seq' ] = this_id                                                        
        msg[ 'type' ] = 'request'                                                     
        self.sendMessage(msg)
    
    def close_connection(self):
        self.recvMessage("Cleanup")
        self.mysocket.close()
    
    def reset_connection(self):
        self.close_connection()
        self.mysocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.connect()
        
        

## clientClass

### clientClass provides method APIs for the following DAP requests

The parameters are also shown with default values for them, if any

- **attach_to_server()** - method API for the **attach** request.
- **initialize(clientID=None)** - method API for the **initialize** request.
- **set_breakpoints(line_list)** - method API for the **set_breakpoints** request. This takes in a list of line numbers where to set the breakpoints. To delete all breakpoints, send an empty list. To remove one particular breakpoint, send a list with all but that line number. Each call to set_breakpoints updates the list of breakpoints.
- **configurationDone** - method API for the **configurationDone** request.
- **pause(threadID=1)** - method API for the **pause** request. This takes in the threadID which to pause. Default is 1.
- **next_step(threadID=1)** - method API for the **next** request. This takes in the threadID on which to do "next". Default is 1.
- **continue_execution(threadID=1)** - method API for the **continue** request. This takes in the threadID which to pause. Default is 1.
- **get_threads** - method API for the **threads** request.
- **get_stack_trace(threadID=1)** - method API for the **stacktrace** request. This takes in the threadID which to pause. Default is 1.
- **get_scopes(frameID)** - method API for the **scopes** request. This takes in the frameID of the stacktrace whose variables we are to observe.
- **get_variables(variable_ref)** - method API for the **variables** request. This takes in the variablesReference value of the variable whose value we are to observe.
- **step_in** - method API for the **stepIn** request. This takes in the threadID in which to "stepIn" to. Default is 1. Note that if there's nothing to step into at a paused instance, this behaves identical to "next".
- **step_out** - method API for the **stepOut** request. This takes in the threadID in which to "stepOut" of. Default is 1. Note that if the frame is at the topmost level function, this behaves a bit weirdly, so take care when to call step_out.
- **read_server(cleanup=False)** - method to read from the server without sending any request. If cleanup is True, closes the socket after the read.

**Note** - For set_breakpoints, next_step, continue_execution, get_stack_trace, get_scopes, get_variables, the thread we are querying needs to be paused. Threads are paused (or stopped in DAP's lingo) due to one of these reasons (and some more) -      'step', 'breakpoint', 'exception', 'pause', 'entry', 'goto', 'function breakpoint', 'data breakpoint', etc.


In [90]:
class clientClass:   
    
    def __init__(self, filename, hostname='localhost', port=5678, timeout=5):
        self.dbgfile = filename
        self.connection = connectionClass(hostname, port, timeout)
      
    def attach_to_server(self):
        self.connection.doRequest( 
            {'command': 'attach',                                                      
             'arguments': {                                                           
               'name': 'client_class',                                                         
             },  
            } 
        )                                                                          
        self.connection.recvMessage("attach")
    
    def initialize(self, clientID=None):
        if clientID is None:
            clientID = self.__class__.__name__
        self.connection.doRequest( {                                                                    
            'command': 'initialize',                                                  
            'arguments': {                                                            
                'adapterID': 'pydevd',
                'clientID': clientID,
                'clientName':'client.py',                                                  
                'linesStartAt1': True,                                                  
                'columnsStartAt1': True,                                                
                'pathFormat': 'path',
                'supportsVariableType': True,
                'supportsVariablePaging': False,
                'supportsRunInTerminalRequest': True
            },                                                                        
        } )
        self.connection.recvMessage("initialize")
    
    def set_breakpoints(self, line_list):
        bp_list = []
        for line in line_list:
            bp_list.append({'line':line})
        
        self.initialize()
        
        self.connection.doRequest( {                                                                    
              'command': 'setBreakpoints',                                              
              'arguments': {                                                            
                'source':{'path': self.dbgfile}, #could alternately give sourceReference.
                'breakpoints':bp_list,                                                                                                        
              },                                                                        
        } )
        self.connection.recvMessage("setBreakpoints")
        
        self.configurationDone()

    def configurationDone(self):
        self.connection.doRequest( {                                                                    
              'command': 'configurationDone',                                                                                                                         
            } ) 
        self.connection.recvMessage("configurationDone")
        
    def pause(self, threadID=1):
        #The pause request suspends the debuggee.
        #Not sure when to use it
        self.connection.doRequest( {                                                                    
              'command': 'pause',                                                  
              'arguments': {                                                            
                'threadId': threadID,                                                  
              },                                                                        
            } )                                                                         
        self.connection.recvMessage("pause")
    
    def next_step(self, threadID=1):
        #Only makes sense when execution of the debuggee is paused.
        #executes one step and then pauses again
        self.connection.doRequest( {                                                                    
            'command': 'next',
                'arguments': {                                                            
            'threadId': threadID,                                                          
            },                                                                        
        } )
        self.connection.recvMessage("next")
    
    def continue_execution(self, threadID=1):
        #Only makes sense when execution of the debuggee is paused.
        #executes one step and then pauses again
        self.connection.doRequest( {                                                                    
            'command': 'continue',
                'arguments': {                                                            
            'threadId': threadID,                                                          
            },                                                                        
        } )
        self.connection.recvMessage("continue")
    
    def get_threads(self):
        self.connection.DoRequest( {                                                                    
          'command': 'threads',                                                     
        } )
        self.connection.recvMessage("threads")
    
    def get_stack_trace(self, threadID=1):
        self.connection.doRequest( {                                                                    
          'command': 'stackTrace',
                'arguments': {                                                            
            'threadId': threadID,                                                          
          },                                                                        
        } )
        self.connection.recvMessage("stackTrace")
    
    def get_scopes(self, frameID):
        self.connection.doRequest( {                                                                    
          'command': 'scopes',
            'arguments': {                                                            
            'frameId': frameID,                                                          
          },                                                                        
        } )
        self.connection.recvMessage("scopes")
    
    def get_variables(self, variable_ref):
        self.connection.doRequest( {                                                                    
            'command': 'variables',
                'arguments': {                                                            
                    'variablesReference': variable_ref,
          },                                                                        
        } )
        self.connection.recvMessage("variables")  
        
    def step_in(self, threadID=1):
        self.connection.doRequest( {                                                                    
            'command': 'stepIn',
                'arguments': {                                                            
                    'threadId': threadID,
                },                                                                        
        } )
        self.connection.recvMessage("stepIn")
    
    def step_out(self, threadID=1):
        self.connection.doRequest( {                                                                    
            'command': 'stepOut',
                'arguments': {                                                            
                    'threadId': threadID,
                },                                                                        
        } )
        self.connection.recvMessage("stepOut")
    
    def read_server(self, cleanup=False):
        if cleanup:
            self.connection.close_connection()
        else:
            self.connection.recvMessage("End of execution")

# Testing the clientClass and connectionClass on a sample test file

## myUser.py
We will be using the file myUser.py as a debuggee. Here is a view on what myUser.py contains

In [118]:
#Code duplication because I wasn't sure how exactly to import it from the fuzzingbook package.
#improved upon the fuzzingbook code with line numbers. maybe push this in?

from pygments import highlight, lexers, formatters
from pygments.lexers import get_lexer_for_filename

#will figure this out.
def print_file(filename, lexer=None):
    contents = open(filename, "rb").read().decode('utf-8')
    if lexer is None:
        lexer = get_lexer_for_filename(filename)
    
    colorful_contents = highlight(contents, lexer, formatters.TerminalFormatter())
    contents_with_line_no = colorful_contents.split("\n")
    no_of_lines = len(contents_with_line_no)
    size_of_linesno = len(str(no_of_lines))

    for i in range(no_of_lines):
        contents_with_line_no[i] = ('{0:' + str(size_of_linesno) + '}').format(i) + " " + contents_with_line_no[i]
    contents_with_line_no = '\n'.join(contents_with_line_no)
    print(contents_with_line_no, end="")

In [119]:
print_file('myUser.py')

 0 class User:
 1     def __init__(self, name, email):
 2         self._name = name
 3         self._email = email
 4     
 5     def get_name(self):
 6         return self._name
 7     
 8     def get_email(self):
 9         return self._email
10         
11     def do_something(self):
12         print ("Hello, I'm " + str(self))
13         
14     def __str__(self):
15         return self._name + ", " + self._email
16 
17 
18 users_list = [User("test1", "test1@mail.com"), User("test2", "test2@notmail.com"), User("test3", "test3@somewhatmail.com")]
19 
20 for user in users_list:
21     user.do_something()
22 

Pretty straightforward. There is a "User" class that contains two instance variables named "\_name" and "\_email" that can be accessed using the "get_name" and "get_email' methods and there's "do_something" method that prints out the user's name and email in a formatted string. There is a list named "users_list" that contain 2 User instances. 

## Extracting variable values

We wish to extract the names and emails of both the User instances in the "users_list" list.


For this, we set a breakpoint on line no 21 in myUser.py

### Each of the following executions have been discussed and explained in the cell before it.

### Object instantiation
We create an instance of clientClass called myClass. At creation, myClass creates an instance of connectionClass within itself.
That instance established a connection with the debugger. If this is the first time the debugger is being connected to, it responds with a message detailing the DAP output event that has been triggered. Otherwise, it doesn't respond anything and our socket read times out after 5 seconds

In [93]:
myClass = clientClass("myUser.py")

Received data:
Content-Length: 130
{'body': {'category': 'telemetry',
          'data': {'version': '4.3.2'},
          'output': 'ptvsd'},
 'event': 'output',
 'seq': 0,
 'type': 'event'}



### Attaching to the debugger

We have just opened a connection to the server, we need to attach our client to the debugger. We do this by calling the **attach_to_server** method of myClass.

We observe that the response to the "attach" request has been successful (success=True) in the first response. 

We also observe that ptvsd sends multiple responses to our debug client, each with different details. The second response is that "process" event has happened. This event indicates that the debugger has begun debugging a new process. Either one that it has launched, or one that it has attached to.

The final response by the server is the internal reference number of the thread that has been started. It is 1 in this case. It is always 1 when there a single threaded process is running.

In [94]:
myClass.attach_to_server()

Sending data: Content-Length: 91
{'arguments': {'name': 'client_class'},
 'command': 'attach',
 'seq': 109,
 'type': 'request'}

Received data:
Content-Length: 115
{'body': {},
 'command': 'attach',
 'message': '',
 'request_seq': 109,
 'seq': 1,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 151
{'body': {'isLocalProcess': True,
          'name': 'myUser.py',
          'startMethod': 'attach',
          'systemProcessId': 615},
 'event': 'process',
 'seq': 2,
 'type': 'event'}

Received data:
Content-Length: 92
{'body': {'reason': 'started', 'threadId': 1},
 'event': 'thread',
 'seq': 3,
 'type': 'event'}



### Setting breakpoints

Here we set breakpoints. This message is to be always preceded by the "initialise" request and always followed by the "configurationDone" request. Without the "initialise" method, the debugger won't set the breakpoints, and without the "configurationDone" request, it won't continue execution.

Hence, our first message is "initialise". We receive 2 responses. The first notifies the client what requests is allowed with this initialise method. The second confirms that the process has been initialised.

Our second message is "setBreakpoints". In this case, we receive one response with the list of lines where the breakpoints have been set.

Our third message is "configurationDone". This request evokes one response that the request was successful.

After that the debugger continues execution (it always does that when we send the "configurationDone" request). And in its execution, the stopped event happens because it hits the breakpoints we just set. This is the final message.

**N.B. To remove breakpoints, execute set_breakpoints with an empty list**

In [95]:
myClass.set_breakpoints([21])

Sending data: Content-Length: 319
{'arguments': {'adapterID': 'pydevd',
               'clientID': 'clientClass',
               'clientName': 'client.py',
               'columnsStartAt1': True,
               'linesStartAt1': True,
               'pathFormat': 'path',
               'supportsRunInTerminalRequest': True,
               'supportsVariablePaging': False,
               'supportsVariableType': True},
 'command': 'initialize',
 'seq': 110,
 'type': 'request'}

Received data:
Content-Length: 867
{'body': {'exceptionBreakpointFilters': [{'default': False,
                                          'filter': 'raised',
                                          'label': 'Raised Exceptions'},
                                         {'default': True,
                                          'filter': 'uncaught',
                                          'label': 'Uncaught Exceptions'}],
          'supportTerminateDebuggee': True,
          'supportsCompletionsRequest': True,
   

### Getting breakpoint location.

**The thread, whose variable we are interested in, needs to be in "stopped" state before we can execute the following command**

Since the debuggee is already stopped, we can query where is it stopped.
Please note that in the previous cell, when the process/thread was stopped, it didn't respond with the line number of where it stopped.

We can only get that info by a "stacktrace" request. This request will also be used as you will see in the following sections, for other purposes, especially for getting variable values and, as the name suggests, getting other details of the stack.

We see that our debuggee program "myUser.py" has the stack frame id '2' (because it is running as a module in debug_server.ipynb).
We also notice that the stack is currently on line no 21, which is where we had st a breakpoint. In case of multiple breakpoints, we will get an update of where the line currently is.

In [96]:
myClass.get_stack_trace(1)

Sending data: Content-Length: 86
{'arguments': {'threadId': 1},
 'command': 'stackTrace',
 'seq': 113,
 'type': 'request'}

Received data:
Content-Length: 158
{'body': {'module': {'id': 0,
                     'name': '__main__',
                     'path': '/home/jovyan/work/HiWi/myUser.py'},
          'reason': 'new'},
 'event': 'module',
 'seq': 9,
 'type': 'event'}

Received data:
Content-Length: 157
{'body': {'module': {'id': 1,
                     'name': 'runpy',
                     'path': '/opt/conda/lib/python3.6/runpy.py'},
          'reason': 'new'},
 'event': 'module',
 'seq': 10,
 'type': 'event'}

Received data:
Content-Length: 287
{'body': {'stackFrames': [{'column': 1,
                           'id': 2,
                           'line': 21,
                           'name': '<module>',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}}],
          'totalFrames': 1},
 'comm

### Step in and Step out

**The thread, whose variable we are interested in, needs to be in "stopped" state before we can execute the following command**

- As mentioned in the description of clientClass, if there is nothing to step into at a given point, step_in behaves identical to "next". We will observe this phenomenon and also what happens when we step into a function and how they both are different.

- As mentioned in the description of clientClass, if there is nothing to step out of at a given point, step_out behaves weirdly (meaning I haven't been able to figure out exactly what's happening and I don't know what is the correct expected behaviour), so please be careful when stepping out

Since we know that the process is stopped at line no 21, which is a for loop check, step_in won't step into something but just behave like "next", which we will see later in a different section.

In [97]:
myClass.step_in(1)

Sending data: Content-Length: 82
{'arguments': {'threadId': 1},
 'command': 'stepIn',
 'seq': 114,
 'type': 'request'}

Received data:
Content-Length: 116
{'body': {},
 'command': 'stepIn',
 'message': '',
 'request_seq': 114,
 'seq': 12,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 13,
 'type': 'event'}

Received data:
Content-Length: 146
{'body': {'allThreadsStopped': True,
          'preserveFocusHint': False,
          'reason': 'step',
          'threadId': 1},
 'event': 'stopped',
 'seq': 14,
 'type': 'event'}



- We will see in the next command that since there was nothing to step into, step_in behaved exactly like a "next" command.

In [98]:
myClass.get_stack_trace(1)

Sending data: Content-Length: 86
{'arguments': {'threadId': 1},
 'command': 'stackTrace',
 'seq': 115,
 'type': 'request'}

Received data:
Content-Length: 287
{'body': {'stackFrames': [{'column': 1,
                           'id': 2,
                           'line': 22,
                           'name': '<module>',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}}],
          'totalFrames': 1},
 'command': 'stackTrace',
 'message': '',
 'request_seq': 115,
 'seq': 15,
 'success': True,
 'type': 'response'}



- Line 22 contains a call to "do_something" method of the User class, something we can step into. So, we run step_in again and observe the output of stack_trace.

In [99]:
myClass.step_in(1)

Sending data: Content-Length: 82
{'arguments': {'threadId': 1},
 'command': 'stepIn',
 'seq': 116,
 'type': 'request'}

Received data:
Content-Length: 116
{'body': {},
 'command': 'stepIn',
 'message': '',
 'request_seq': 116,
 'seq': 16,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 17,
 'type': 'event'}

Received data:
Content-Length: 146
{'body': {'allThreadsStopped': True,
          'preserveFocusHint': False,
          'reason': 'step',
          'threadId': 1},
 'event': 'stopped',
 'seq': 18,
 'type': 'event'}



- We see in the next command that now we have an additional frame since we stepped into the "do_something" method. This has the frame id value 3.

In [100]:
myClass.get_stack_trace(1)

Sending data: Content-Length: 86
{'arguments': {'threadId': 1},
 'command': 'stackTrace',
 'seq': 117,
 'type': 'request'}

Received data:
Content-Length: 425
{'body': {'stackFrames': [{'column': 1,
                           'id': 3,
                           'line': 13,
                           'name': 'do_something',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}},
                          {'column': 1,
                           'id': 2,
                           'line': 22,
                           'name': '<module>',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}}],
          'totalFrames': 2},
 'command': 'stackTrace',
 'message': '',
 'request_seq': 117,
 'seq': 19,
 'success': True,
 'type': 'response'}



#### Now we step out of this do_something method and observe the stack trace again.

We could have continued within the "do_something" frame with the "next" command but it has only one line of code so that would have been equivalent to stepping out.

In [101]:
myClass.step_out(1)

Sending data: Content-Length: 83
{'arguments': {'threadId': 1},
 'command': 'stepOut',
 'seq': 118,
 'type': 'request'}

Received data:
Content-Length: 117
{'body': {},
 'command': 'stepOut',
 'message': '',
 'request_seq': 118,
 'seq': 20,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 21,
 'type': 'event'}

Received data:
Content-Length: 146
{'body': {'allThreadsStopped': True,
          'preserveFocusHint': True,
          'reason': 'pause',
          'threadId': 1},
 'event': 'stopped',
 'seq': 22,
 'type': 'event'}



- If you go observe the debug_server.ipynb, we see that one of the users' details have been printed.
- We also see in the output of the stackTrace command in the following cell that it has exited the do_something method's frame.

In [102]:
myClass.get_stack_trace(1)

Sending data: Content-Length: 86
{'arguments': {'threadId': 1},
 'command': 'stackTrace',
 'seq': 119,
 'type': 'request'}

Received data:
Content-Length: 287
{'body': {'stackFrames': [{'column': 1,
                           'id': 2,
                           'line': 22,
                           'name': '<module>',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}}],
          'totalFrames': 1},
 'command': 'stackTrace',
 'message': '',
 'request_seq': 119,
 'seq': 23,
 'success': True,
 'type': 'response'}



### Getting variables

**The thread, whose variable we are interested in, needs to be in "stopped" state before we can execute the following commands**

Getting variables is trickier than the other commands we have executed so far. Attaching was a single command, setting breakpoints required 3 commands. Getting variable details could require many more.

We can't directly query the debug server for the variable details (because the server itself doesn't have the data to find out the values).  To get variables, we need the scope frame i.e. the frame of the stack which contains the variables. 

But we also can't directly query the server for the frame as the server doesn't hold this data in itself at this point. To get the stack frame, we need to query for the list of stacks. Once we ask the server for a copy of the stacks, the server reads and returns the values, as well as stores the state locally which can now be queried.

 - So we first query for the stack. We execute this using the 'stackTrace' command that returns all the stack frames.
 
 We get similar response as the previous sections. Here, we will use this information to extract details about variables

In [103]:
myClass.get_stack_trace(1)

Sending data: Content-Length: 86
{'arguments': {'threadId': 1},
 'command': 'stackTrace',
 'seq': 120,
 'type': 'request'}

Received data:
Content-Length: 287
{'body': {'stackFrames': [{'column': 1,
                           'id': 2,
                           'line': 22,
                           'name': '<module>',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}}],
          'totalFrames': 1},
 'command': 'stackTrace',
 'message': '',
 'request_seq': 120,
 'seq': 24,
 'success': True,
 'type': 'response'}



- Now we query for the frame details using the frameid '2'.

The request returns the variable scopes for a given stackframe ID, which we can use to query for our variables.

In [104]:
myClass.get_scopes(2)

Sending data: Content-Length: 81
{'arguments': {'frameId': 2},
 'command': 'scopes',
 'seq': 121,
 'type': 'request'}

Received data:
Content-Length: 205
{'body': {'scopes': [{'expensive': False,
                      'name': 'Locals',
                      'source': {},
                      'variablesReference': 2}]},
 'command': 'scopes',
 'message': '',
 'request_seq': 121,
 'seq': 25,
 'success': True,
 'type': 'response'}



- We query using the variableReference 2.

The variablesReference is a required parameter whose value we can only derive using "stackTrace" and then the "scopes" request. For now, we explore what the variableReference 2 contains. Passing any other value now, will result in a failure response from the server.

We see that the variable contains a lot of details. The variableReference 3 contains details about the "User" class. Our variable of interest, the "users_list" list has the variableReference 4. We use this value to query further for the contents of the users_list list.

In [105]:
myClass.get_variables(2)

Sending data: Content-Length: 95
{'arguments': {'variablesReference': 2},
 'command': 'variables',
 'seq': 122,
 'type': 'request'}

Received data:
Content-Length: 2515
{'body': {'variables': [{'evaluateName': 'User',
                         'name': 'User',
                         'type': 'type',
                         'value': "<class '__main__.User'>",
                         'variablesReference': 4},
                        {'evaluateName': 'user',
                         'name': 'user',
                         'type': 'User',
                         'value': '<__main__.User object at 0x7f762d204cc0>',
                         'variablesReference': 5},
                        {'evaluateName': 'users_list',
                         'name': 'users_list',
                         'type': 'list',
                         'value': '[<__main__.User objec...62d204cc0>, '
                                  '<__main__.User objec...62d204e48>, '
                                  '<__ma

- Upon querying with the variableReference 6, we see that the three members of the list have the variableReference 5, 9 and 10 respectively

In [106]:
myClass.get_variables(6)

Sending data: Content-Length: 95
{'arguments': {'variablesReference': 6},
 'command': 'variables',
 'seq': 123,
 'type': 'request'}

Received data:
Content-Length: 719
{'body': {'variables': [{'evaluateName': 'users_list[0]',
                         'name': '0',
                         'type': 'User',
                         'value': '<__main__.User object at 0x7f762d204cc0>',
                         'variablesReference': 5},
                        {'evaluateName': 'users_list[1]',
                         'name': '1',
                         'type': 'User',
                         'value': '<__main__.User object at 0x7f762d204e48>',
                         'variablesReference': 9},
                        {'evaluateName': 'users_list[2]',
                         'name': '2',
                         'type': 'User',
                         'value': '<__main__.User object at 0x7f762d204e80>',
                         'variablesReference': 10},
                        {'evaluat

- Upon querying variableReference 5, we get the details of the first user instance, which is what we wanted all this while.

In [107]:
myClass.get_variables(5)

Sending data: Content-Length: 95
{'arguments': {'variablesReference': 5},
 'command': 'variables',
 'seq': 124,
 'type': 'request'}

Received data:
Content-Length: 481
{'body': {'variables': [{'evaluateName': 'users_list[0]._email',
                         'name': '_email',
                         'presentationHint': {'attributes': ['rawString']},
                         'type': 'str',
                         'value': "'test1@mail.com'",
                         'variablesReference': 0},
                        {'evaluateName': 'users_list[0]._name',
                         'name': '_name',
                         'presentationHint': {'attributes': ['rawString']},
                         'type': 'str',
                         'value': "'test1'",
                         'variablesReference': 0}]},
 'command': 'variables',
 'message': '',
 'request_seq': 124,
 'seq': 28,
 'success': True,
 'type': 'response'}



- Similarly, querying with variableReference 9 and 10 gives us the details of the second and third instances of Users respectively

In [108]:
myClass.get_variables(9)

Sending data: Content-Length: 95
{'arguments': {'variablesReference': 9},
 'command': 'variables',
 'seq': 125,
 'type': 'request'}

Received data:
Content-Length: 484
{'body': {'variables': [{'evaluateName': 'users_list[1]._email',
                         'name': '_email',
                         'presentationHint': {'attributes': ['rawString']},
                         'type': 'str',
                         'value': "'test2@notmail.com'",
                         'variablesReference': 0},
                        {'evaluateName': 'users_list[1]._name',
                         'name': '_name',
                         'presentationHint': {'attributes': ['rawString']},
                         'type': 'str',
                         'value': "'test2'",
                         'variablesReference': 0}]},
 'command': 'variables',
 'message': '',
 'request_seq': 125,
 'seq': 29,
 'success': True,
 'type': 'response'}



In [109]:
myClass.get_variables(10)

Sending data: Content-Length: 96
{'arguments': {'variablesReference': 10},
 'command': 'variables',
 'seq': 126,
 'type': 'request'}

Received data:
Content-Length: 489
{'body': {'variables': [{'evaluateName': 'users_list[2]._email',
                         'name': '_email',
                         'presentationHint': {'attributes': ['rawString']},
                         'type': 'str',
                         'value': "'test3@somewhatmail.com'",
                         'variablesReference': 0},
                        {'evaluateName': 'users_list[2]._name',
                         'name': '_name',
                         'presentationHint': {'attributes': ['rawString']},
                         'type': 'str',
                         'value': "'test3'",
                         'variablesReference': 0}]},
 'command': 'variables',
 'message': '',
 'request_seq': 126,
 'seq': 30,
 'success': True,
 'type': 'response'}



### We continue to try the other methods we have in our client class - next and continue



### The "next" request

Now we try to run the next command, since we have hit a breakpoint, we see what happens next.
We notice that the execution stops after one step because that is how "next" works in all debuggers.
Notice the difference for the reason value. Here it is "step". When it hit a breakpoint, after we set the breakpoint, it was "breakpoint"

The server returns 3 responses. The first one is a direct response of our request, which was successful here.
The second describes the event that happened because of our request, which is "continued".
The third describes the event "stopped" because it was done with the one step it had to execute.


It is now stopped on line 21 -> right before the call to do_something() for the second member of users_list

In [110]:
#The next request starts the debuggee to run again for one step.
#only works the debuggee is stopped
myClass.next_step(1)

Sending data: Content-Length: 80
{'arguments': {'threadId': 1}, 'command': 'next', 'seq': 127, 'type': 'request'}

Received data:
Content-Length: 114
{'body': {},
 'command': 'next',
 'message': '',
 'request_seq': 127,
 'seq': 31,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 32,
 'type': 'event'}

Received data:
Content-Length: 146
{'body': {'allThreadsStopped': True,
          'preserveFocusHint': False,
          'reason': 'step',
          'threadId': 1},
 'event': 'stopped',
 'seq': 33,
 'type': 'event'}



- It is now stopped on line 21-> right before the call to do_something() for the second member of users_list. We confirm this by sending a StackTrace request.

In [111]:
myClass.get_stack_trace(1)

Sending data: Content-Length: 86
{'arguments': {'threadId': 1},
 'command': 'stackTrace',
 'seq': 128,
 'type': 'request'}

Received data:
Content-Length: 287
{'body': {'stackFrames': [{'column': 1,
                           'id': 2,
                           'line': 21,
                           'name': '<module>',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}}],
          'totalFrames': 1},
 'command': 'stackTrace',
 'message': '',
 'request_seq': 128,
 'seq': 34,
 'success': True,
 'type': 'response'}



- If we execute "next" twice more, we will see that do_something() will print the details fo the second member in the debug_server.ipynb file.

Indeed, check the debug_server.ipynb before executing the next command and after executing it and you will notice the difference.

In [112]:
#The next request starts the debuggee to run again for one step.
#only works the debuggee is stopped
myClass.next_step(1)

Sending data: Content-Length: 80
{'arguments': {'threadId': 1}, 'command': 'next', 'seq': 129, 'type': 'request'}

Received data:
Content-Length: 114
{'body': {},
 'command': 'next',
 'message': '',
 'request_seq': 129,
 'seq': 35,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 36,
 'type': 'event'}

Received data:
Content-Length: 146
{'body': {'allThreadsStopped': True,
          'preserveFocusHint': False,
          'reason': 'step',
          'threadId': 1},
 'event': 'stopped',
 'seq': 37,
 'type': 'event'}



In [113]:
#The next request starts the debuggee to run again for one step.
#only works the debuggee is stopped
myClass.next_step(1)

Sending data: Content-Length: 80
{'arguments': {'threadId': 1}, 'command': 'next', 'seq': 130, 'type': 'request'}

Received data:
Content-Length: 114
{'body': {},
 'command': 'next',
 'message': '',
 'request_seq': 130,
 'seq': 38,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 39,
 'type': 'event'}

Received data:
Content-Length: 146
{'body': {'allThreadsStopped': True,
          'preserveFocusHint': False,
          'reason': 'step',
          'threadId': 1},
 'event': 'stopped',
 'seq': 40,
 'type': 'event'}



In [114]:
myClass.get_stack_trace(1)

Sending data: Content-Length: 86
{'arguments': {'threadId': 1},
 'command': 'stackTrace',
 'seq': 131,
 'type': 'request'}

Received data:
Content-Length: 287
{'body': {'stackFrames': [{'column': 1,
                           'id': 2,
                           'line': 21,
                           'name': '<module>',
                           'source': {'path': '/home/jovyan/work/HiWi/myUser.py',
                                      'sourceReference': 0}}],
          'totalFrames': 1},
 'command': 'stackTrace',
 'message': '',
 'request_seq': 131,
 'seq': 41,
 'success': True,
 'type': 'response'}



#### The continue request

We now try the continue command

The server again replies with 3 responses. The first and the second response is same as the "next" request's responses. The third one here is slightly different because it is the "stopped" event due to the breakpoint we had set, on line 21.

We see, on debug_server.ipynb notebook that the details of the second User is also printed. However the execution isn't completed now, because even though there were just 3 members in the users_list, it will execute the check one more time before exiting the for loop.

In [115]:
myClass.continue_execution(1)

Sending data: Content-Length: 84
{'arguments': {'threadId': 1},
 'command': 'continue',
 'seq': 132,
 'type': 'request'}

Received data:
Content-Length: 145
{'body': {'allThreadsContinued': True},
 'command': 'continue',
 'message': '',
 'request_seq': 132,
 'seq': 42,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 43,
 'type': 'event'}

Received data:
Content-Length: 152
{'body': {'allThreadsStopped': True,
          'preserveFocusHint': False,
          'reason': 'breakpoint',
          'threadId': 1},
 'event': 'stopped',
 'seq': 44,
 'type': 'event'}



 - **Now we wrap up the execution by sending the "continue" request once more.**
 
 Notice that the debug_server that we started on debug_server.ipynb is still running. Once we do this, it will continue adn finish the execution (since there aren't any more breakpoints in its further execution)

In [116]:
myClass.continue_execution(1)

Sending data: Content-Length: 84
{'arguments': {'threadId': 1},
 'command': 'continue',
 'seq': 133,
 'type': 'request'}

Received data:
Content-Length: 145
{'body': {'allThreadsContinued': True},
 'command': 'continue',
 'message': '',
 'request_seq': 133,
 'seq': 45,
 'success': True,
 'type': 'response'}

Received data:
Content-Length: 104
{'body': {'allThreadsContinued': True, 'threadId': 1},
 'event': 'continued',
 'seq': 46,
 'type': 'event'}



#### Final read of the server. 

We finish this demo with a final read of the server, for any messages it may have sent. This will also close the socket on our end. If the socket server is already closed, it will return nothing.

In [117]:
myClass.read_server(cleanup=True)